In [ ]:
import sqlite3
import pandas as pd
import numpy as np

Build a profile of providers referring patients to the major hospitals in Nashville. Are certain specialties more likely to refer to a particular hospital over the others?

In [ ]:
# We decided to define "Major Hospitals" somewhat narrowly, those with a display_name from the taxonomy table of 
# "General Acute Care Hospitals"
# This excludes children's hospitals, critical access hospitals, psychiatric hospitals, etc.


#creating a query to return the npi and organization name for all Nashville hospitals
query = """
SELECT npi, organization_name 
FROM npidata
LEFT JOIN taxonomy
USING(taxonomy_code)
WHERE entity_type_code = 2
AND location_address_city_name = 'NASHVILLE'
AND location_address_state_name = 'TN'
AND display_name = 'General Acute Care Hospital'
"""

with sqlite3.connect('../data/hop_db.sqlite') as db: 
    nash_hosp = pd.read_sql(query, db)
    
db.close()

In [ ]:
# Now pulling all providers that refer to Nashville hospitals
# First, set my Nashville hospital query as a CTE, put aliases in place for clearer column names
# Then join the CTE to the hop table on the to_npi column to get only the referrals to these hospitals
# Then bring in details about the providers making the referrals from the npi_data and taxonomy tables

query = """
WITH nash_hosp AS (
    SELECT npi AS hosp_npi, organization_name AS hospital
    FROM npidata
    LEFT JOIN taxonomy
    USING(taxonomy_code)
    WHERE entity_type_code = 2
    AND location_address_city_name = 'NASHVILLE'
    AND location_address_state_name = 'TN'
    AND classification = 'General Acute Care Hospital'
)
SELECT transaction_count, 
       hospital,
       location_address_city_name, 
       location_address_state_name, 
       location_address_postal_code, 
       display_name AS specialty, 
       from_npi, 
       first_name,
       last_name,
       credential,
       patient_count,
       average_day_wait
FROM hop
INNER JOIN npidata
ON from_npi = npi
INNER JOIN nash_hosp
ON to_npi = hosp_npi
LEFT JOIN taxonomy
USING(taxonomy_code)
WHERE transaction_count >= 50
AND average_day_wait < 50
AND entity_type_code = 1
"""

with sqlite3.connect('../data/hop_db.sqlite') as db: 
    nash_hosp_referrers = pd.read_sql(query, db)
    
db.close()

In [ ]:
# Number of unique providers referring to Nashville hospitals by state

(
    nash_hosp_referrers
    .groupby('location_address_state_name')['from_npi']
    .nunique()
    .to_frame()
    .reset_index()
    .sort_values('from_npi', ascending = False)
)

In [ ]:
# For the providers within TN, the number referring from different cities

(
    nash_hosp_referrers
    .loc[nash_hosp_referrers['location_address_state_name'] == 'TN']
    .groupby('location_address_city_name')['from_npi']
    .nunique()
    .to_frame()
    .reset_index()
    .sort_values('from_npi', ascending = False)
    .head(36)
)

#### Exploring the top providers in terms of referrals for each hospital

In [ ]:
nash_hosp_referrers['hosp_top_referrals'] = (
    nash_hosp_referrers
    .groupby('hospital')['transaction_count']
    .rank(ascending = False)
)

# Looking at the top 10 for each hospital
nash_hosp_referrers.loc[nash_hosp_referrers['hosp_top_referrals'] <= 10].sort_values(by= ['hospital', 'hosp_top_referrals'])

#### Now taking a look referrals at the specialty level, which hospitals they refer to most

In [ ]:
# create a df that indicates the number of referrals (transactions) made by practitioners of a given specialty to a given hospital
referring_specialties = (
    nash_hosp_referrers
    .groupby(['specialty', 'hospital'])['transaction_count']
    .sum()
    .to_frame()
    .reset_index()
    .sort_values(by = ['specialty', 'transaction_count'], ascending = [True, False])
)

# rank the hospitals based on number of referrals for each specialty, rank of 1 means most referrals
referring_specialties['rank'] = (
    referring_specialties
    .groupby('specialty')['transaction_count']
    .rank(ascending = False)
)

# create a column that creates a percent based comparison referrals within a given specialty
# null values in this column indicate there is no hospital receiving fewer referrals
# smaller values here indicate that a much smaller percentage of referrals are going to the hospital with the next higher rank
referring_specialties['percent_for_next'] = (
    round(100 * referring_specialties.groupby('specialty')['transaction_count'].shift(-1) 
          / 
          referring_specialties['transaction_count'], 2)
)

referring_specialties

In [ ]:
# Quick look at how many times hospitals rank #1 for referrals from different specialties
referring_specialties.loc[referring_specialties['rank'] == 1.0]['hospital'].value_counts()

In [ ]:
# now looking for hospitals ranked #1 and sorting to see those that either have NO competition
# meaning no other hospitals receive referrals from this specialty, or where there is a step drop in the number of 
# referrals to the #2 ranked hospital (at least a 25% drop)
(
    referring_specialties
    .loc[(referring_specialties['rank'] == 1.0)
        &
        ((referring_specialties['percent_for_next'] < 75)
        | 
         (referring_specialties['percent_for_next'].isna()))]
    .sort_values('percent_for_next', na_position = 'first')
    .head(60)
)

#### Broadening the specialty definitions

In [ ]:
query = """
WITH nash_hosp AS (
    SELECT npi AS hosp_npi, organization_name AS hospital
    FROM npidata
    LEFT JOIN taxonomy
    USING(taxonomy_code)
    WHERE entity_type_code = 2
    AND location_address_city_name = 'NASHVILLE'
    AND location_address_state_name = 'TN'
    AND classification = 'General Acute Care Hospital'
)
SELECT from_npi, 
       first_name,
       last_name,
       credential, 
       location_address_city_name, 
       location_address_state_name, 
       location_address_postal_code, 
       classification AS specialty, 
       display_name AS sub_specialty,
       transaction_count,
       patient_count,
       average_day_wait,
       hospital
FROM hop
INNER JOIN npidata
ON from_npi = npi
INNER JOIN nash_hosp
ON to_npi = hosp_npi
LEFT JOIN taxonomy
USING(taxonomy_code)
WHERE transaction_count >= 50
AND average_day_wait < 50
AND entity_type_code = 1
"""

with sqlite3.connect('../data/hop_db.sqlite') as db: 
    nash_hosp_referrers_b = pd.read_sql(query, db)
    
db.close()

In [ ]:
nash_hosp_referrers_b 

In [ ]:
# create a df that indicates the number of referrals (transactions) made by practitioners of a given specialty to a given hospital
referring_specialties_b = (
    nash_hosp_referrers_b
    .groupby(['specialty', 'hospital'])['transaction_count']
    .sum()
    .to_frame()
    .reset_index()
    .sort_values(by = ['specialty', 'transaction_count'], ascending = [True, False])
)

# rank the hospitals based on number of referrals for each specialty, rank of 1 means most referrals
referring_specialties_b['rank'] = (
    referring_specialties_b
    .groupby('specialty')['transaction_count']
    .rank(ascending = False)
)

# create a column that creates a percent based comparison referrals within a given specialty
# null values in this column indicate there is no hospital receiving fewer referrals
# smaller values here indicate that a much smaller percentage of referrals are going to the hospital with the next higher rank
referring_specialties_b['percent_for_next'] = (
    round(100 * referring_specialties_b.groupby('specialty')['transaction_count'].shift(-1) 
          / 
          referring_specialties_b['transaction_count'], 2)
)

referring_specialties_b

In [ ]:
(
    referring_specialties_b
    .loc[(referring_specialties_b['rank'] == 1.0)
        &
        ((referring_specialties_b['percent_for_next'] < 75)
        | 
         (referring_specialties_b['percent_for_next'].isna()))]
    .sort_values('percent_for_next', na_position = 'first')
)

Determine which professionals Vanderbilt Hospital should reach out to in the Nashville area to expand their own patient volume.   
    - First, research which professionals are sending significant numbers of patients only to competitor hospitals (such as TriStar Centennial Medical Center).  
    - Next, consider the specialty of the provider. If Vanderbilt wants to increase volume from Orthopedic Surgeons or from Family Medicine doctors who should they reach out to in those areas?

In [ ]:
(len(referring_specialties['specialty'].unique()) 
 - 
 len(referring_specialties.loc[referring_specialties['hospital'] == 'VANDERBILT UNIVERSITY MEDICAL CENTER'])
)
# of specialties (by display_name) that do not refer to Vanderbilt is 8

In [ ]:
(
    referring_specialties
    .loc[(referring_specialties['rank'] == 1.0)
        & 
         (referring_specialties['percent_for_next'].isna())
        &
        (referring_specialties['hospital'] != 'VANDERBILT UNIVERSITY MEDICAL CENTER')]
    .sort_values('percent_for_next', na_position = 'first')
)

In [ ]:
(len(referring_specialties_b['specialty'].unique()) 
 - 
 len(referring_specialties_b.loc[referring_specialties_b['hospital'] == 'VANDERBILT UNIVERSITY MEDICAL CENTER'])
)
# of specialties (by classification) that do not refer to Vanderbilt is 3

In [ ]:
(
    referring_specialties_b
    .loc[(referring_specialties_b['rank'] == 1.0)
        & 
         (referring_specialties_b['percent_for_next'].isna())
        &
        (referring_specialties_b['hospital'] != 'VANDERBILT UNIVERSITY MEDICAL CENTER')]
    .sort_values('percent_for_next', na_position = 'first')
)

In [ ]:
vandy_ref = nash_hosp_referrers.loc[nash_hosp_referrers['hospital'].str.contains("Vanderbilt")]['from_npi'].unique()

In [ ]:
# top 40 providers (in terms of total referrals) NOT referring to Vandy at all
(
    nash_hosp_referrers
    .loc[(~nash_hosp_referrers['from_npi'].isin(vandy_ref))
         &
         (nash_hosp_referrers['location_address_state_name'] == 'TN')]
    .groupby(['from_npi', 'location_address_city_name', 'specialty'])
    ['transaction_count'].sum()
    .to_frame()
    .reset_index()
    .sort_values('transaction_count', ascending = False)
    .head(40)
)
    

In [ ]:
# Creating a query to search for practitioners by specialty and see the number of referrals they are sending to hospitals
# other than Vandy

# Using the broader definition of specialties (classification) here
specialty = 'Family Medicine'

(
    nash_hosp_referrers_b
    .loc[(~nash_hosp_referrers_b['hospital'].str.contains('Vanderbilt'))
         &
         (nash_hosp_referrers_b['specialty'] == f'{specialty}')]
    .groupby(['from_npi', 'location_address_state_name', 'location_address_city_name', 'specialty'])
    ['transaction_count'].sum()
    .to_frame()
    .reset_index()
    .sort_values('transaction_count', ascending = False)
)

In [ ]:
#import pickle

#with open('data/nash_hosp_referrers_b.pickle', 'wb') as file:
#    pickle.dump(nash_hosp_referrers_b, file)

In [ ]:
#with open('data/nash_hosp_referrers.pickle', 'wb') as file:
#    pickle.dump(nash_hosp_referrers, file)